In [77]:
import time
import random
import requests
from urllib.parse import quote  # 新增這行

class CnyesNewsSpider():
    
    def get_newslist_info(self, keyword="新光金", page=1):
        """ 獲取新聞列表資訊

        :param keyword: 搜尋關鍵字
        :param page: 頁數
        :return newslist_info: 新聞資料
        """
        headers = {
            'Origin': 'https://news.cnyes.com/',
            'Referer': 'https://news.cnyes.com/',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        }
        
        # 對關鍵字進行 URL 編碼
        encoded_keyword = quote(keyword)
        url = f"https://api.cnyes.com/media/api/v1/search?q={encoded_keyword}&page={page}"
        r = requests.get(url, headers=headers)
        
        if r.status_code != requests.codes.ok:
            print('請求失敗', r.status_code)
            return None
        
        newslist_info = r.json()['items']
        return newslist_info


if __name__ == "__main__":
    cnyes_news_spider = CnyesNewsSpider()
    
    # 可以在這裡輸入不同的股票名稱，例如：台積電、鴻海、大立光
    keyword = input("請輸入股票名稱（例如：新光金、台積電）：")
    newslist_info = cnyes_news_spider.get_newslist_info(keyword=keyword)
    
    if newslist_info:
        for news in newslist_info["data"]:
            print(f'    ------------ {news["newsId"]} ------------')
            print(f'    新聞 > 標題：{news["title"]}')
            print(f'    新聞 > URL：https://news.cnyes.com/news/id/{news["newsId"]}')
            print(f'    新聞 > 發布時間：{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(news["publishAt"]))}')
            print()
    
    # 如果要爬取多頁新聞，建議加入隨機 delay 一段時間，避免被反爬蟲偵測
    # time.sleep(random.uniform(2, 5))


請輸入股票名稱（例如：新光金、台積電）： 台積電


    ------------ 5882543 ------------
    新聞 > 標題：盤中速報 - 費城半導體大漲1.01%，報4621.5點
    新聞 > URL：https://news.cnyes.com/news/id/5882543
    新聞 > 發布時間：2025-03-04 22:53:46

    ------------ 5882324 ------------
    新聞 > 標題：〈美股早盤〉川普關稅引發報復 主要指數齊開低
    新聞 > URL：https://news.cnyes.com/news/id/5882324
    新聞 > 發布時間：2025-03-04 22:46:51

    ------------ 5882237 ------------
    新聞 > 標題：跟著<mark>台積電</mark>！大魯閣美國運動娛樂中心亮相 未來每年開10家店
    新聞 > URL：https://news.cnyes.com/news/id/5882237
    新聞 > 發布時間：2025-03-04 19:19:36

    ------------ 5882232 ------------
    新聞 > 標題：聯策、邑昇加入半導體國家隊大聯盟 要打群架搶占市場先機
    新聞 > URL：https://news.cnyes.com/news/id/5882232
    新聞 > 發布時間：2025-03-04 19:03:34

    ------------ 5882207 ------------
    新聞 > 標題：<mark>台積電</mark>加碼美國投資定案 鉅亨買基金：台股撥雲見日迎利多
    新聞 > URL：https://news.cnyes.com/news/id/5882207
    新聞 > 發布時間：2025-03-04 18:34:32

    ------------ 5882153 ------------
    新聞 > 標題：<mark>台積電</mark> 兩百萬 鴻海30萬 起點?【新建
    新聞 > URL：https://news.cnyes.com/news/id/5882153
    新聞 > 發布時間：2

In [107]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

class OfficialSpider():

    def spider_func(self, number):
        # 獲取今昨天日期
        today = datetime.today().strftime('%Y-%m-%d')
        yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
        # 設定 Selenium 瀏覽器
        options = Options()
        options.add_argument("--headless")  # 無頭模式，不顯示瀏覽器
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        # 啟動 WebDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        
        # 目標網址
        url = f"https://goodinfo.tw/tw/StockAnnounceList.asp?PAGE=1&START_DT={yesterday}&END_DT={today}&STOCK_ID={number}&KEY_WORD=&NEWS_SRC=%E5%85%AC%E5%91%8A%E8%A8%8A%E6%81%AF"
        
        # 進入頁面
        driver.get(url)
        
        # 等待 JavaScript 加載
        time.sleep(5)  # 這裡可以視情況調整
        
        # 取得頁面 HTML
        html = driver.page_source
        
        # 關閉瀏覽器
        driver.quit()
        
        # 解析 HTML
        soup = BeautifulSoup(html, "lxml")
        
        # 找到公告訊息的表格
        table = soup.find("table", class_="p4_4 row_bg_2n row_mouse_over")
        if table:
            rows = table.find_all("tr")
            for row in rows:
                link_tag = row.find("a", class_="link_black")
                date_tag = row.find("span", style="font-size:9pt;color:gray;font-weight:normal;")
                if link_tag and date_tag:
                    title = link_tag.text.strip()
                    link = "https://goodinfo.tw/tw/" + link_tag["href"]
                    date = date_tag.text.strip().replace("(公告訊息", "").replace(")", "")
                    
                    print(f"公告標題: {title}")
                    print(f"公告日期: {date}")
                    print(f"公告連結: {link}\n")
        else:
            print("未找到公告訊息表格！")

if __name__ == "__main__":
    o_spider = OfiicialSpider()
    
    # 可以在這裡輸入不同的股票名稱，例如：台積電、鴻海、大立光
    number = input("請輸入股票代號（例如：2330）：")
    news = o_spider.spider_func(number)
    

請輸入股票代號（例如：2330）： 2888


公告標題: 新光金舉辦113年第四季法人說明會
公告日期:  2025/03/04 15:04
公告連結: https://goodinfo.tw/tw/StockAnnounceDetail.asp?STOCK_ID=2888&CLAIM_TIME=2025%2F03%2F04+15%3A04%3A29



In [105]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

class ETSpider():
    
    def spider_func(self, number):
        # 獲取今昨天日期
        today = datetime.today().strftime('%Y-%m-%d')
        yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
        # 設定 Selenium 瀏覽器
        options = Options()
        options.add_argument("--headless")  # 無頭模式，不顯示瀏覽器
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        # 啟動 WebDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        
        # 目標網址
        url = f"https://goodinfo.tw/tw/StockAnnounceList.asp?PAGE=1&START_DT={yesterday}&END_DT={today}&STOCK_ID={number}&KEY_WORD=&NEWS_SRC=ETtoday%E6%96%B0%E8%81%9E%E9%9B%B2"
        
        # 進入頁面
        driver.get(url)
        
        # 等待 JavaScript 加載
        time.sleep(5)  # 這裡可以視情況調整
        
        # 取得頁面 HTML
        html = driver.page_source
        
        # 關閉瀏覽器
        driver.quit()
        
        # 解析 HTML
        soup = BeautifulSoup(html, "lxml")
        
        # 找到公告訊息的表格
        table = soup.find("table", class_="p4_4 row_bg_2n row_mouse_over")
        if table:
            rows = table.find_all("tr")
            for row in rows:
                link_tag = row.find("a", class_="link_black")
                date_tag = row.find("span", style="font-size:9pt;color:gray;font-weight:normal;")
                if link_tag and date_tag:
                    title = link_tag.text.strip()
                    link = "https://goodinfo.tw/tw/" + link_tag["href"]
                    date = date_tag.text.strip().replace("(公告訊息", "").replace(")", "")
                    
                    print(f"公告標題: {title}")
                    print(f"公告日期: {date}")
                    print(f"公告連結: {link}\n")
        else:
            print("未找到公告訊息表格！")

if __name__ == "__main__":
    et_spider = ETSpider()
    
    # 可以在這裡輸入不同的股票名稱，例如：台積電、鴻海、大立光
    number = input("請輸入股票代號（例如：2330）：")
    news = et_spider.spider_func(number)

請輸入股票代號（例如：2330）： 2330


公告標題: 台積電赴美投資千億美元「是雙贏談判」　郭明錤提五重點分析
公告日期: (ETtoday新聞雲 2025/03/04 19:05
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919398

公告標題: 台積電加碼投資美國　台股跌幅居亞洲第二
公告日期: (ETtoday新聞雲 2025/03/04 18:53
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919389

公告標題: 郭智輝樂觀看台積電加碼對美投資　「我們來支持他」
公告日期: (ETtoday新聞雲 2025/03/04 18:48
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919380

公告標題: 台積電加碼對美投資　商總許舒博憂「放緩在台佈局」
公告日期: (ETtoday新聞雲 2025/03/04 18:31
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919378

公告標題: 台積電、鴻海成最大苦主！外資連2日賣超　台股遭提款破千億
公告日期: (ETtoday新聞雲 2025/03/04 18:12
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919274

公告標題: 台積電加碼投資3.3兆未來隱憂　「這是和川普談判的示範」
公告日期: (ETtoday新聞雲 2025/03/04 18:11
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919359

公告標題: 台積電失守千元散戶不敢玩！　零股交易倒退嚕
公告日期: (ETtoday新聞雲 2025/03/04 18:03
公告連結: https://goodinfo.tw/tw/https://finance.ettoday.net/news/2919275

公告標題: 盤點「護國神山」進駐產業園區　寶山房價5年飆漲1.2倍居冠
公

In [115]:
import time
import random
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager




class CnyesNewsSpider():
    
    def get_newslist_info(self, keyword="新光金", page=1):
        """ 獲取新聞列表資訊

        :param keyword: 搜尋關鍵字
        :param page: 頁數
        :return newslist_info: 新聞資料
        """
        headers = {
            'Origin': 'https://news.cnyes.com/',
            'Referer': 'https://news.cnyes.com/',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        }
        
        # 對關鍵字進行 URL 編碼
        encoded_keyword = quote(keyword)
        url = f"https://api.cnyes.com/media/api/v1/search?q={encoded_keyword}&page={page}"
        r = requests.get(url, headers=headers)
        
        if r.status_code != requests.codes.ok:
            print('請求失敗', r.status_code)
            return None
        
        newslist_info = r.json()['items']
        return newslist_info
        
class OfficialSpider():

    def spider_func(self, number):
        # 獲取今昨天日期
        today = datetime.today().strftime('%Y-%m-%d')
        yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
        # 設定 Selenium 瀏覽器
        options = Options()
        options.add_argument("--headless")  # 無頭模式，不顯示瀏覽器
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        # 啟動 WebDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        
        # 目標網址
        url = f"https://goodinfo.tw/tw/StockAnnounceList.asp?PAGE=1&START_DT={yesterday}&END_DT={today}&STOCK_ID={number}&KEY_WORD=&NEWS_SRC=%E5%85%AC%E5%91%8A%E8%A8%8A%E6%81%AF"
        
        # 進入頁面
        driver.get(url)
        
        # 等待 JavaScript 加載
        time.sleep(5)  # 這裡可以視情況調整
        
        # 取得頁面 HTML
        html = driver.page_source
        
        # 關閉瀏覽器
        driver.quit()
        
        # 解析 HTML
        soup = BeautifulSoup(html, "lxml")
        
        # 找到公告訊息的表格
        table = soup.find("table", class_="p4_4 row_bg_2n row_mouse_over")
        if table:
            rows = table.find_all("tr")
            for row in rows:
                link_tag = row.find("a", class_="link_black")
                date_tag = row.find("span", style="font-size:9pt;color:gray;font-weight:normal;")
                if link_tag and date_tag:
                    title = link_tag.text.strip()
                    link = "https://goodinfo.tw/tw/" + link_tag["href"]
                    date = date_tag.text.strip().replace("(公告訊息", "").replace(")", "")
                    
                    print(f"公告標題: {title}")
                    print(f"公告日期: {date}")
                    print(f"公告連結: {link}\n")
        else:
            print("未找到公告訊息表格！")


    
class ETSpider():
    
    def spider_func(self, number):
        # 獲取今昨天日期
        today = datetime.today().strftime('%Y-%m-%d')
        yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
        # 設定 Selenium 瀏覽器
        options = Options()
        options.add_argument("--headless")  # 無頭模式，不顯示瀏覽器
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        # 啟動 WebDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        
        # 目標網址
        url = f"https://goodinfo.tw/tw/StockAnnounceList.asp?PAGE=1&START_DT={yesterday}&END_DT={today}&STOCK_ID={number}&KEY_WORD=&NEWS_SRC=ETtoday%E6%96%B0%E8%81%9E%E9%9B%B2"
        
        # 進入頁面
        driver.get(url)
        
        # 等待 JavaScript 加載
        time.sleep(5)  # 這裡可以視情況調整
        
        # 取得頁面 HTML
        html = driver.page_source
        
        # 關閉瀏覽器
        driver.quit()
        
        # 解析 HTML
        soup = BeautifulSoup(html, "lxml")
        
        # 找到公告訊息的表格
        table = soup.find("table", class_="p4_4 row_bg_2n row_mouse_over")
        if table:
            rows = table.find_all("tr")
            for row in rows:
                link_tag = row.find("a", class_="link_black")
                date_tag = row.find("span", style="font-size:9pt;color:gray;font-weight:normal;")
                if link_tag and date_tag:
                    title = link_tag.text.strip()
                    link = "https://goodinfo.tw/tw/" + link_tag["href"]
                    date = date_tag.text.strip().replace("(公告訊息", "").replace(")", "")
                    
                    print(f"公告標題: {title}")
                    print(f"公告日期: {date}")
                    print(f"公告連結: {link}\n")
        else:
            print("未找到公告訊息表格！")
    

if __name__ == "__main__":
    # 使用者輸入
    user_input = input("請輸入股票名稱與代號（格式: 股票名稱 股票代號）：")
    
    # 使用 split 拆分
    parts = user_input.rsplit(' ', 1)  # 只切割最後一個空格，確保名稱中有空格時仍可正確處理
    
    # 取得股票名稱與代號
    if len(parts) == 2:
        stock_name, stock_id = parts
        print(f"股票名稱: {stock_name}")
        print(f"股票代號: {stock_id}")
    else:
        print("輸入格式錯誤，請使用『股票名稱 股票代號』的格式")

    #鉅亨新聞
    cnyes_news_spider = CnyesNewsSpider()
    newslist_info = cnyes_news_spider.get_newslist_info(keyword=stock_name)
    
    if newslist_info:
        for news in newslist_info["data"]:
            print(f'    ------------ {news["newsId"]} ------------')
            print(f'    新聞 > 標題：{news["title"]}')
            print(f'    新聞 > URL：https://news.cnyes.com/news/id/{news["newsId"]}')
            print(f'    新聞 > 發布時間：{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(news["publishAt"]))}')
            print()

    #公告訊息
    o_spider = OfficialSpider()
    o_news = o_spider.spider_func(stock_id)

    #ETtoday新聞
    et_spider = ETSpider()
    et_news = et_spider.spider_func(stock_id)

請輸入股票名稱與代號（格式: 股票名稱 股票代號）： 台積電 2330


股票名稱: 台積電
股票代號: 2330
    ------------ 5895565 ------------
    新聞 > 標題：Meta開始測試其首款自有AI訓練用晶片
    新聞 > URL：https://news.cnyes.com/news/id/5895565
    新聞 > 發布時間：2025-03-11 19:53:23

    ------------ 5894130 ------------
    新聞 > 標題：台股跌幅收斂拉下影線 黃文清：短震後 瞄準這3類股搶跌深反彈
    新聞 > URL：https://news.cnyes.com/news/id/5894130
    新聞 > 發布時間：2025-03-11 19:51:43

    ------------ 5895359 ------------
    新聞 > 標題：路透：Meta與<mark>台積電</mark>合作測試自研AI晶片
    新聞 > URL：https://news.cnyes.com/news/id/5895359
    新聞 > 發布時間：2025-03-11 19:39:24

    ------------ 5895340 ------------
    新聞 > 標題：台股殺聲隆隆跌幅居亞股第二慘 金管會向市場信心喊話
    新聞 > URL：https://news.cnyes.com/news/id/5895340
    新聞 > 發布時間：2025-03-11 19:21:06

    ------------ 5895339 ------------
    新聞 > 標題：晶心科看好四大應用 今年權利金盼顯著成長
    新聞 > URL：https://news.cnyes.com/news/id/5895339
    新聞 > 發布時間：2025-03-11 19:17:58

    ------------ 5895336 ------------
    新聞 > 標題：0050奪回定期定額人氣王 K值小於20正是獲利進場點？
    新聞 > URL：https://news.cnyes.com/news/id/5895336
    新聞 > 發布時間：2025-03-11 19:1